In [5]:
from Sentiment_analysis import *
import ast
import string
import sklearn.feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('trainRooms.csv')
# Clone aspectSentences because TFIDF and Sentiment scoring requires different formats of strings
train['aspectSentences2'] = train['aspectSentences'].copy()
train['aspectSentences'] = train['aspectSentences'].apply(ast.literal_eval)

In [7]:
# To get balanced dataset
mins = train['aspectRating'].value_counts().min()
train_balanced = train.groupby("aspectRating").apply(lambda x: x.sample(n=mins))

In [8]:
# Get sentiment score
train_balanced = train_balanced.apply(addSentimentScores, axis = 1)
X_senti = train_balanced.iloc[:,6:]

In [9]:
def custom_tokenizer(review):  
    review = review.encode('ascii',errors='ignore')
    review = review.translate(None,string.punctuation)
    words = nltk.tokenize.word_tokenize(review)
    words = [nltk.stem.porter.PorterStemmer().stem(word) for word in words]
    # stop words removed by vectorizer
    
    return words

In [10]:
TfidVect = sklearn.feature_extraction.text.TfidfVectorizer(min_df=10, ngram_range=(1,2), tokenizer = custom_tokenizer,
                                                      stop_words = nltk.corpus.stopwords.words('english'))
X_tfidf = TfidVect.fit_transform(train_balanced['aspectSentences2'].tolist())
X_tfidf = X_tfidf.toarray()

In [11]:
newX = np.concatenate((X_senti,X_tfidf), axis = 1)

In [12]:
lin_clf = sklearn.svm.LinearSVC()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(newX, train_balanced['aspectRating'].tolist() )
lin_clf.fit(X_train,y_train)
y_predicted = lin_clf.predict(X_test)

In [14]:
print (pd.Series(y_predicted) == pd.Series(y_test)).mean()
print ((pd.Series(y_predicted)-pd.Series(y_test))**2).mean()

0.417662938681
1.62051677594
